In [ ]:
# Homework 1 (due 6/28/2024)
# Credit Statement: Kyrylo Bakumenko

In [ ]:
import ipywidgets as widgets # lil ipython widgets for UX
from IPython.display import display, clear_output

# initialize board
board = [[" " for _ in range(3)] for _ in range(3)]
current_player = 'X' # first player is X
# default player/computer assignment
player_marker = 'X'
computer_marker = 'O'
memo = {} # memo for dp search for computer

# check if board is in a 'win' state
def check_winner(mark):
    win_conditions = [
        [board[0][0], board[0][1], board[0][2]],
        [board[1][0], board[1][1], board[1][2]],
        [board[2][0], board[2][1], board[2][2]],
        [board[0][0], board[1][0], board[2][0]],
        [board[0][1], board[1][1], board[2][1]],
        [board[0][2], board[1][2], board[2][2]],
        [board[0][0], board[1][1], board[2][2]],
        [board[2][0], board[1][1], board[0][2]],
    ]
    return [mark, mark, mark] in win_conditions

# find empty (available) spots
def get_empty_positions():
    positions = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == " ":
                positions.append((i, j))
    return positions

# Minimax Algorith, computer move helper
def minimax(board, depth, is_maximizing, memo):

    # convert board list to a string to use as a dictionary key
    board_key = ''.join(board[0]) + ''.join(board[1]) + ''.join(board[2])
    # add key for who is moving
    board_key += 'T' if is_maximizing else 'F'

    # check if in hashmap
    if board_key in memo:
        return memo[board_key]

    # base cases
    if check_winner(computer_marker):
        return 1
    if check_winner(player_marker):
        return -1
    if not get_empty_positions():
        return 0

    # recursive case, alternate between computer and simulated opponent moves
    if is_maximizing:
        best_score = -float('inf')
        for pos in get_empty_positions():
            board[pos[0]][pos[1]] = computer_marker
            score = minimax(board, depth + 1, False, memo)
            board[pos[0]][pos[1]] = " "
            best_score = max(score, best_score)
        memo[board_key] = best_score
        return best_score
    else:
        best_score = float('inf')
        for pos in get_empty_positions():
            board[pos[0]][pos[1]] = player_marker
            score = minimax(board, depth + 1, True, memo)
            board[pos[0]][pos[1]] = " "
            best_score = min(score, best_score)
        memo[board_key] = best_score
        return best_score

def user_move(button):
    global current_player
    row, col = map(int, button.tooltip.split(','))
    if board[row][col] == " ":
        board[row][col] = player_marker
        button.style.button_color = '#444'
        button.disabled = True
        button.description = player_marker
        if check_winner(player_marker):
            display(widgets.Label("Congratulations! You win!"))
            disable_buttons()
            return
        current_player = computer_marker
        computer_move()

def computer_move():
    global memo
    best_score = -float('inf')
    best_move = None

    # iterate through all available spots for computer
    # record best score
    for pos in get_empty_positions():
        board[pos[0]][pos[1]] = computer_marker
        score = minimax(board, 0, False, memo)
        board[pos[0]][pos[1]] = " "
        if score > best_score:
            best_score = score
            best_move = pos

    # update for computer move
    if best_move:
        board[best_move[0]][best_move[1]] = computer_marker
        button = buttons[best_move[0]][best_move[1]]
        button.description = computer_marker
        button.style.button_color = '#444'
        button.disabled = True

    # check if win or draw
    if check_winner(computer_marker):
        display(widgets.Label("Computer wins! Better luck next time."))
        disable_buttons()
        return
    current_player = player_marker

    if not get_empty_positions():
        display(widgets.Label("It's a draw!"))

def disable_buttons():
    for row in buttons:
        for button in row:
            button.disabled = True

def restart_game(button):
    global board, current_player, buttons, player_marker, computer_marker
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = 'X'
    for row in buttons:
        for button in row:
            button.description = ""
            button.style.button_color = None
            button.disabled = False
    choose_marker()

# choose marker (replaces y/n question on moving first)
def choose_marker():
    print("X moves first! O moves second!")
    def set_marker(button):
        global player_marker, computer_marker, current_player
        player_marker = button.description
        computer_marker = 'O' if player_marker == 'X' else 'X'
        current_player = player_marker
        start_game()

    button_x = widgets.Button(description="X", layout=widgets.Layout(width="60px", height="60px"))
    button_o = widgets.Button(description="O", layout=widgets.Layout(width="60px", height="60px"))
    button_x.on_click(set_marker)
    button_o.on_click(set_marker)

    display(widgets.Label("Choose your marker:"))
    display(widgets.HBox([button_x, button_o]))

def start_game():
    clear_output()
    display(board_ui)
    display(restart_button)
    if player_marker == 'O':
        computer_move()

### board setup ###
# buttons
buttons = [[widgets.Button(description="", tooltip=f"{row},{col}", layout=widgets.Layout(width="60px", height="60px"))
            for col in range(3)] for row in range(3)]

# click events
for row in buttons:
    for button in row:
        button.on_click(user_move)

# board display
board_ui = widgets.VBox([widgets.HBox(row) for row in buttons])
restart_button = widgets.Button(description="Restart Game", layout=widgets.Layout(width="188px", height="30px"))
restart_button.on_click(restart_game)
### board setupo end ###

choose_marker()

X moves first! O moves second!


Label(value='Choose your marker:')